## A pipeline for processing and analyzing multiplexed images

#### Related project: A spatial single-cell type map of adult human spermatogenesis (Cecilia Bergström group)

### Import required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import io

# stardist
from stardist.models import StarDist2D
from stardist.plot import render_label
from csbdeep.utils import normalize

from skimage import io, filters, measure, segmentation, color, util
from skimage.filters import threshold_otsu

import warnings
warnings.filterwarnings('ignore')


2023-08-14 16:28:13.743880: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Defined functions

In [2]:
def nonzero_intensity_mean(mask: np.ndarray, img: np.ndarray) -> float:
    data = img[mask]
    data = data[data != 0]
    
    if data.size != 0:
        return np.mean(data)
    else:
        return 0
    
def export_table_to_dataframe(tables, cols):
    tables = [pd.DataFrame(table) for table in tables] # create dataframe for each table
    tables = [table.set_index('label') for table in tables] # reset segmentation label as table index
    
    mean_intens = pd.concat(tables, axis=1)
    mean_intens.columns = cols
    return mean_intens

def getMeanIntensity(ref_image, labels):
    images = [ref_img[(x),:,:] for x in range(ref_img.shape[0])]

    properties = ['label', 'intensity_mean']
    
    tables = [measure.regionprops_table(labels, image, properties=properties)
              for image in images]
    mean_intens = export_table_to_dataframe(tables, cols)
    
    return mean_intens

def getNonZeroMeanIntensity(ref_image, labels, path, outpath):
    images = [ref_img[(x),:,:] for x in range(ref_img.shape[0])] # for each channel
    
    # get enlarged-labels image as a binary mask
    enlarged_binary_mask = enlarged_labels.copy()
    enlarged_binary_mask[enlarged_binary_mask > 0] = 1

    tables = []
    i = 0
    for img in images:

        filteredByNucleiMask = enlarged_binary_mask * img # for each channel, exclude regions that are outside of the cellular region defined by the segmentation segmentation

        if i == 0: # nuclei image, copy original labeled mask
            thresholded = enlarged_binary_mask

        elif i==2: # 520 channel, read mask from Ilastik segmentation

            # load mask image
            label520 = io.imread(os.path.join(path, ilastik_mask))
            thresholded = (label520 == 1) * 1

        else:
            thr = threshold_otsu(filteredByNucleiMask)
            thresholded = (img >= thr) * 1

        io.imsave(outpath + '/filteredByNucleiMask_' + cols[i] + '.tif', filteredByNucleiMask)
        io.imsave(outpath + '/filteredByNucleiMask_intens_' + cols[i] + '.tif', filteredByNucleiMask*thresholded)
        io.imsave(outpath + '/binary_' + cols[i] + '.tif', util.img_as_ubyte(thresholded*255))

        nonzero_intensity_means = measure.regionprops_table(labels, filteredByNucleiMask*thresholded, properties=['label'], extra_properties=[nonzero_intensity_mean])
        tables.append(nonzero_intensity_means)
        i=i+1

    mean_intens = export_table_to_dataframe(tables, cols)
    
    return mean_intens

### Define input path, image of interest and other parameters

In [3]:
# define input path
inputpath = '/Users/giselemiranda/ToOneDrive/BIIF/projects/Feria_Cecilia/input/' # example '/Users/projects/'
runBioEngine = False

# channel sequence
cols = ['DAPI','OPAL480','OPAL520','OPAL570','OPAL620','OPAL690','OPAL780','Autofluorescence']

### Run batch

In [4]:
for imgID in os.listdir(inputpath):
    subdir = os.path.join(inputpath, imgID)
    
    if os.path.isdir(subdir):
        
        ref_image = ''
        ilastik_mask = ''
    
        # process subdirectory to get input file and ilastik mask
        for file in os.listdir(subdir):
            if 'component_data.tif' in file:
                ref_image = file
            if 'Simple Segmentation.tiff' in file:
                ilastik_mask = file

        # if files are found, then continue processing
        if len(ref_image) > 0 and len(ilastik_mask) > 0:

            # create output path
            outpath = os.path.join(subdir, 'output')
            if not os.path.exists(outpath):
                os.makedirs(outpath)

            # read reference image
            ref_img = io.imread(os.path.join(subdir, ref_image))

            # select DAPI channel
            nuclei = ref_img[0,:,:]
            print("nuclei: loaded ", nuclei.shape)

            # run StarDist
            model = StarDist2D.from_pretrained('2D_versatile_fluo') # load pretrained model
            labels, _ = model.predict_instances(normalize(nuclei)) # get predictions for nuclei 

            # get binary mask
            binary_mask = labels.copy()
            binary_mask[binary_mask > 0] = 1

            # save labels and binary
            io.imsave(outpath + '/DAPI_labels.tif',labels)
            io.imsave(outpath + '/DAPI_binary.tif',util.img_as_ubyte(binary_mask*255))

            # expand labels to incorporate cells' neighborhoods
            enlarged_labels = segmentation.expand_labels(labels, distance=3)
            io.imsave(outpath + '/DAPI_labels_enlarged.tif',enlarged_labels)

            # get mean fluorescence intensity for each nuclei and for each channel
            mean_intens = getMeanIntensity(ref_image, labels)
            mean_intens.to_csv(os.path.join(outpath, 'mean_intensity.csv'), sep=';')

            # get mean fluorescence intensity for each nuclei and for each channel, given the thresholded masks
            mean_intens = getNonZeroMeanIntensity(ref_image, labels, subdir, outpath)
            mean_intens.to_csv(os.path.join(outpath, 'mean_intensity_threshold.csv'), sep=';')
        

nuclei: loaded  (3996, 3996)
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
1/1 [==============================] - 8s 8s/step
nuclei: loaded  (3996, 3996)
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.
1/1 [==============================] - 7s 7s/step
